[Home](../../README.md)

### Data Preprocessing

This is a demonstration of data preprocessing using [Pandas](https://pandas.pydata.org/) the library for data analysis and manipulation.

This Jupyter Notepad is less about steps and more about different processes you can apply to your data before data wrangling. For this demonstration we will use relatively a complex real dataset that compares health measures with the speed of progress of type 2 adult onset diabetes.

> [!Note]
> None of these processes are destructive to the source CSV.

#### Load the required dependencies

In [7]:
# Import frameworks
import pandas as pd

####  Store the data as a local variable

The data frame is a Pandas object that structures your tabular data into an appropriate format. It loads the complete data in memory so it is now ready for preprocessing.

In [51]:
data_frame = pd.read_csv("2.1.1.Diabeties_Sample_Data.csv")

#### Dealing with null values

Null values during data analysis can cause runtime errors and unexpected results. It is important to identify null values and deal with them appropriately before training a model.

The `isnull().sum()` method call returns the null values in any column.

In [ ]:
data_frame.isnull().sum()

If you have null data there are many ways to deal with the empty/null values. These are the two most common approaches.
1. Remove any row with a null value with a `dropna()` method call.
2. Replace missing values with another value with a `fillna()` method call. Generally, we use mean value for numerical columns because it may cause minimal changes in your mathematical analysis while maintaining the original size of the data.

Students should reflect why this example removes the null 'SEX' but replacing the mean 'AGE'?

In [58]:
# Remove Null values
data_frame = data_frame.dropna(subset=['SEX'])
data_frame.isnull().sum()

AGE                  1
SEX                  0
BMI                  0
BP                   0
Total Cholesterol    0
Blood Sugar Level    0
Target               0
dtype: int64

In [ ]:
# Replace Null values with the mean value for the column
data_frame['AGE'] = data_frame['AGE'].fillna(data_frame['AGE'].mean())
data_frame.isnull().sum()

#### Remove Duplicates

Duplicate data can have detrimental effects on your machine learning models and outcomes, such as reducing data diversity and representativeness, which can lead to overfitting or biased models.

The `duplicated().sum()` method call returns the count of duplicate rows in the data frame.

In [ ]:
data_frame.duplicated().sum()

The `drop_duplicates()` method call can be then stored back onto the data_frame variable removing the duplicates.

In [ ]:
data_frame = data_frame.drop_duplicates()
data_frame.duplicated().sum()

#### Replace data

We can run a lambda function on a column to modify its values. For a simple example, let’s convert the Sex to lowercase. To run a function over a complete column, we can use the apply method which iterates over each row and modifies the values.

In [59]:
data_frame['SEX'] = data_frame['SEX'].apply(lambda x: x.lower())
data_frame['SEX'].head()

0    female
1      male
2    female
3      male
4      male
Name: SEX, dtype: object

We can check that there are no data entry errors by the `unique()` method call.

In [60]:
data_frame['SEX'].unique()

array(['female', 'male', 'girl'], dtype=object)

In [62]:
data_frame['SEX'] = data_frame['SEX'].apply(lambda gender: 'male' if gender.lower() == 'male' else 'female')
data_frame['SEX'].unique()

array(['female', 'male'], dtype=object)

#### Filter Your Data

Filtering is like applying the where clause in a database. It is widely used and can help when you need to work on a specific subset of your data. For our use case, let us filter the data to only include rows where the 'SEX' is 'male'. There is no method call for this, we can just use conditional indexing to fulfil our purpose.

In [ ]:
data_frame = data_frame[data_frame['SEX'] == 'male']
data_frame.head()

#### Encoding categorical variables

Data Encoding converts textual data into numerical format, so that it can be used as input for algorithms to process. The reason for encoding is that most machine learning algorithms work with numbers and not with text or categorical variables.

To encode the 'SEX' column you will assigning a number value to the gender. Because the data set only provides 2 values we will use -1 and 1.

In [ ]:
data_frame['SEX'] = data_frame['SEX'].apply(lambda gender: -1 if gender.lower() == 'male' else 1 if gender.lower() == 'female' else None)
print(data_frame['SEX'].head())

#### Remove outliers

Outliers can skew your analysis on numerical columns, and it is important to remove them. We can use the 25th and 75th quartile on numerical data, to get the inter-quartile range. This allows us to estimate an acceptable range, and we can then filter out any values outside this range. Mathematically, outliers are values occurring outside 1.5 times the interquartile range (IQR) from the first quartile (Q1) or third quartile (Q3).

In [ ]:
#get the inter-quartile range on the salary column
print(data_frame['Units Sold'].describe())
Q1 = data_frame['Units Sold'].quantile(0.25)
Q3 = data_frame['Units Sold'].quantile(0.75)
IQR = Q3 - Q1

In [ ]:
# Filter salaries within the acceptable range
data_frame = data_frame[(data_frame['Units Sold'] >= Q1 - 1.5 * IQR) & (data_frame['Units Sold'] <= Q3 + 1.5 * IQR)]
print(data_frame['Units Sold'].describe())

#### Scaling features to a common range



#### Save the preprocessed data to CSV

In [ ]:
data_frame.to_csv('2.1.1.preprocessed_data.csv')